<a href="https://colab.research.google.com/github/dorelysm/NLP_master/blob/main/caracteristicasOdio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Universidad Internacional de La Rioja (UNIR) - Máster Universitario en Inteligencia Artificial - Procesamiento del Lenguaje Natural**

***
Datos del alumno (Nombre y Apellidos): Dorelys Esthafany Martinez Suarez

Fecha: 03/02/25
***

<span style="font-size: 20pt; font-weight: bold; color: #0098cd;">Trabajo: Caracterización de textos</span>

**Objetivos**

Con esta actividad se tratará de que el alumno se familiarice con el manejo de la librería spacy, así como con los conceptos básicos de manejo de las técnicas NER

**Descripción**

En esta actividad debes procesar de forma automática un texto en lenguaje natural para detectar características básicas en el mismo, y para identificar y etiquetar las ocurrencias de conceptos como localización, moneda, empresas, etc.

En la primera parte del ejercicio se proporciona un código fuente a través del cual se lee un archivo de texto y se realiza un preprocesado del mismo. En esta parte el alumno tan sólo debe ejecutar y entender el código proporcionado.

En la segunda parte del ejercicio se plantean una serie de preguntas que deben ser respondidas por el alumno. Cada pregunta deberá responderse con un fragmento de código fuente que esté acompañado de la explicación correspondiente. Para elaborar el código solicitado, el alumno deberá visitar la documentación de la librería spacy, cuyos enlaces se proporcionarán donde corresponda.

# Parte 1: carga y preprocesamiento del texto a analizar

Observa las diferentes librerías que se están importando.

In [3]:
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 14.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import pathlib
import spacy
import pandas as pd
from spacy import displacy
import csv
import es_core_news_md

El siguiente código simplemente carga y preprocesa el texto. Para ello, lo primero que hace es cargar un modelo de lenguaje previamente entrenado. En este caso, se utiliza <i>es_core_news_md</i>:

https://spacy.io/models/es#es_core_news_md


In [5]:
nlp = es_core_news_md.load()

El objeto <i>nlp</i> permite utilizar el modelo de lenguaje cargado, de forma que se puede procesar un texto y obtenerlo en su versión preprocesada. Así, nos permite realizar las diferentes tareas. En este caso, vamos a utilizar el pipeline para hacer un preprocesamiento básico, que consiste en tokenizar el texto.

In [13]:
filename = "./02Dataset_anonimizado.csv"
lines_number = 10000
data = pd.read_csv(filename, encoding='latin1', delimiter=';')

/tmp/ipython-input-71561315.py:3: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename, encoding='latin1', delimiter=';')


El código anterior carga el archivo CSV (opcionalmente con un límite de líneas a leer) y genera la variable <i>data</i>, que contiene un Dataframe (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) con los datos leídos del CSV.

Te vendrá bien conocer la siguiente documentación:
<ul>
    <li>https://spacy.io/api/doc</li>
    <li>https://spacy.io/api/token</li>
    <li>https://spacy.io/api/morphology#morphanalysis</li>
</ul>

### Playground

Utiliza este espacio para hacer pruebas y ensayos con las variables generadas con el código previo. A modo de ejemplo, se ofrece código que realiza las siguientes tareas:


- leer un número dado de líneas del Dataframe y generar dos listas con los valores (se pueden leer directamente del DataFrame, se muestra el ejemplo como una opción más)
- procesar el texto de cada comentario


Para procesarlo, hay utilizar el objeto <i>nlp</i> y así obtener objetos de la clase <i>Doc</i> (https://spacy.io/api/doc)

Visita la documentación de dicha clase y experimenta probando las diferentes funciones y atributos

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575029 entries, 0 to 575028
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   MEDIO                 575029 non-null  object 
 1   SOPORTE               574896 non-null  object 
 2   URL                   574773 non-null  object 
 3   TIPO DE MENSAJE       574767 non-null  object 
 4   CONTENIDO A ANALIZAR  574761 non-null  object 
 5   INTENSIDAD            574630 non-null  float64
 6   TIPO DE ODIO          12293 non-null   object 
 7   TONO HUMORISTICO      143 non-null     object 
 8   MODIFICADOR           350 non-null     object 
 9   Unnamed: 9            0 non-null       float64
 10  Unnamed: 10           0 non-null       float64
 11  Unnamed: 11           0 non-null       float64
 12  Unnamed: 12           0 non-null       float64
 13  Unnamed: 13           0 non-null       float64
 14  Unnamed: 14           0 non-null       float64
 15  

In [15]:
data.head(5)

,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barí§a nunca acaeza ante un segundo b ni an...,3.0,Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.describe()

,INTENSIDAD,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
count,574630.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.078001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.546729,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Puedes insertar aquí código de pruebas para experimentar con las diferentes funciones y atributos de 'doc'.
#print(data["CONTENIDO A ANALIZAR"][1])
#print(data["INTENSIDAD"][1])
doc = []
value = []

#con el bucle, generamos sendas listas con los comentarios ya parseados y con el valor de intensidad
for i in range(0, lines_number):

    #en un primer paso se parsea el comentario. En el segundo paso se añade el objeto a la lista
    tmp_doc = nlp(data["CONTENIDO A ANALIZAR"][i])
    doc.append(tmp_doc)

    #en un primer paso extrae el valor. En el segundo paso se añade el valor a la lista
    tmp_value = data["INTENSIDAD"][i]
    value.append(tmp_value)


#ejemplo de cómo recorrer un comentario palabra por palabra
for token in doc[1]:
    print(token)

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 1.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántos registros contiene el corpus?</span>

In [27]:
# Incluye aquí el código generado para poder responder a tu pregunta
n_registros=(data.shape[0])
print(n_registros)

575029


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 El corpus tiene 575029 registros

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 2.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántas palabras totales hay en los comentarios del corpus?</span>

In [25]:
# Incluye aquí el código generado para poder responder a tu pregunta
data["palabras"]=data["CONTENIDO A ANALIZAR"].str.split().str.len()
suma=data["palabras"].sum()
print(suma)

63012028.0


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 El total de palabras que hay los comentarios del corpus es 63012028

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 3.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál el número promedio de palabras en cada comentario?</span>

In [28]:
# Incluye aquí el código generado para poder responder a tu pregunta
promedio_palabras=suma/n_registros
print(promedio_palabras)

109.58060897798198


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
El promedio de palabras en cada comentario del corpus es de 109.63 palabras por comentario


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 4.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál el número promedio de palabras en los comentarios de cada grupo?</span>

In [30]:
# Incluye aquí el código generado para poder responder a tu pregunta

promedios_por_intensidad = data.groupby('INTENSIDAD').agg({'palabras': 'mean'})
promedios_por_intensidad



,palabras
INTENSIDAD,
0.0,110.586125
1.0,16.819172
2.0,34.364865
3.0,31.831395
4.0,15.292819
5.0,17.085308
6.0,26.615385


In [32]:
data["Odio"] = data["INTENSIDAD"].apply(lambda x: "no odio" if x == 0.0 else "odio")
data.head()

,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,palabras,Odio
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barí§a nunca acaeza ante un segundo b ni an...,3.0,Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,odio
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195.0,no odio
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.0,odio
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,odio
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.0,odio


In [33]:
data.groupby('Odio').agg({'palabras': 'mean'})

,palabras
Odio,
no odio,110.586125
odio,66.443228


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 En promedio los comentarios de no odio tienen un promedio de 110.58 palabras y los de odio tienen un promedio de 66.44 palabras

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 5.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el número promedio de oraciones en los comentarios de cada grupo?</span>

In [40]:
# Incluye aquí el código generado para poder responder a tu pregunta
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Added to download the missing resource
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [43]:
data["n_oraciones"] = data["CONTENIDO A ANALIZAR"].astype(str).apply(lambda x: len(sent_tokenize(x)))

In [44]:
data.groupby('Odio').agg({'n_oraciones': 'mean'})

,n_oraciones
Odio,
no odio,3.784061
odio,2.181489


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 En promedio los comentarios de no odio contienen 3.78 palabras y los de odio 2.18

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 6.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el porcentaje de comentarios que contienen entidades NER en cada grupo?</span>

In [ ]:
# Incluye aquí el código generado para poder responder a tu pregunta
data["NER"]= data["CONTENIDO A ANALIZAR"].apply(lambda x: nlp(x).ents)

<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>


<span style="font-size: 16pt; font-weight: bold; color: #0098cd;">Plantea tus propias preguntas</span>

<span><b>Plantea al menos 4 características</b> del texto cuyo análisis permita una caracterización completa del texto. Puedes utilizar recomendaciones proporcionadas por la IA Generativa, si así lo deseas. Para cada una de las características planteadas, obtén valores separados para los grupos ODIO/NO-ODIO.</span>

<span>En la explicación aportada, deberás <b>explicar el significado de la característica planteada</b> así como la importancia de ésta en la caracterización del texto.</span>

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 1.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">[INDICA AQUÍ LA CARACTERÍSTICA QUE PROPONES]</span>

In [ ]:
# Incluye aquí el código generado para poder responder a tu pregunta


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 2.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">[INDICA AQUÍ LA CARACTERÍSTICA QUE PROPONES]</span>

In [ ]:
# Incluye aquí el código generado para poder responder a tu pregunta


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 3.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">[INDICA AQUÍ LA CARACTERÍSTICA QUE PROPONES]</span>

In [ ]:
# Incluye aquí el código generado para poder responder a tu pregunta


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 4.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">[INDICA AQUÍ LA CARACTERÍSTICA QUE PROPONES]</span>

In [ ]:
# Incluye aquí el código generado para poder responder a tu pregunta


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>


<span style="font-size: 16pt; font-weight: bold; color: #0098cd;">Reflexión final</span>

<span>Una de las utilidades de la caracterización de texto es que nos sirve como etapa de <i>feature-extraction</i> (extración de características) de cara a un posterior sistema de clasificación. Es pertinente, por tanto, reflexionar sobre la capacidad discriminatoria de cada una de las características extraídas. </span>

<span> Responde, para ello, a la siguiente pregunta.</span>

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Reflexión final.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Es posible utilizar alguna de las características extraídas en las preguntas anteriores para determinar si un mensaje contiene odio? Justifica tu respuesta con el análisis estadístico que consideres necesario.</span>

In [ ]:
# Incluye aquí el código generado para poder responder a tu pregunta


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
